In [2]:
import dspy

lm = dspy.LM('openai/gpt-4.1-nano')
dspy.configure(lm=lm)

In [40]:
qa = dspy.Predict('question: str -> response: str')
response = qa(question="what are high memory and low memory on linux?")

print(response.response)

In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of how the kernel manages memory for processes.

- **Low Memory**: This typically refers to the memory that is directly accessible by the kernel and can be used by processes without any special handling. In a 32-bit system, this is usually the first 896 MB of RAM (though this can vary based on the architecture and configuration). Low memory is used for kernel data structures and for user processes that require direct access to memory.

- **High Memory**: This refers to memory that is above the low memory limit and is not directly accessible by the kernel in a 32-bit system. Processes can use this memory, but the kernel must use special mechanisms (like paging) to access it. High memory is often used in systems with large amounts of RAM, allowing more memory to be allocated to user processes while still maintaining a smaller footprint for the kernel.

In 

In [41]:
dspy.inspect_history(n=1)






[2025-05-18T10:39:49.439604]

System message:

Your input fields are:
1. `question` (str)
Your output fields are:
1. `response` (str)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
what are high memory and low memory on linux?

Respond with the corresponding output fields, starting with the field `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## response ## ]]
In Linux, "high memory" and "low memory" refer to different regions of the system's memory address space, particularly in the context of how the kernel manages memory for processes.

- **Low Memory**: This typically refers to the memory that is directly accessible by the kernel and 

In [42]:
cot = dspy.ChainOfThought('question -> response')
r=cot(question="should curly braces appear on their own line?")
r.response

"Curly braces can either appear on their own line or on the same line as the preceding statement, depending on the coding style you choose to follow. It's important to maintain consistency throughout your codebase."

In [43]:
dspy.inspect_history(n=1)






[2025-05-18T10:42:11.263307]

System message:

Your input fields are:
1. `question` (str)
Your output fields are:
1. `reasoning` (str)
2. `response` (str)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `response`.


User message:

[[ ## question ## ]]
should curly braces appear on their own line?

Respond with the corresponding output fields, starting with the field `[[ ## reasoning ## ]]`, then `[[ ## response ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## reasoning ## ]]
The placement of curly braces on their own line is largely a matter of coding style and conventions. In some programming languages and style guides, such as those used in C, Java, and JavaScript, 

In [44]:
import ujson
from dspy.utils import download

# Download question--answer pairs from the RAG-QA Arena "Tech" dataset.
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_examples.jsonl")

with open("ragqa_arena_tech_examples.jsonl") as f:
    data = [ujson.loads(line) for line in f]

In [45]:
data[0]

{'question': 'why igp is used in mpls?',
 'response': "An IGP exchanges routing prefixes between gateways/routers.  \nWithout a routing protocol, you'd have to configure each route on every router and you'd have no dynamic updates when routes change because of link failures. \nFuthermore, within an MPLS network, an IGP is vital for advertising the internal topology and ensuring connectivity for MP-BGP inside the network.",
 'gold_doc_ids': [2822, 2823]}

In [46]:
data = [dspy.Example(**d).with_inputs('question') for d in data]

# Let's pick an `example` here from the data.
example = data[2]
example

Example({'question': 'why are my text messages coming up as maybe?', 'response': 'This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". \n\nHowever, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.', 'gold_doc_ids': [3956, 3957, 8034]}) (input_keys={'question'})

In [47]:
import random

random.Random(0).shuffle(data)
trainset, devset, testset = data[:200], data[200:500], data[500:1000]

len(trainset), len(devset), len(testset)

(200, 300, 500)

In [48]:
example.inputs()

Example({'question': 'why are my text messages coming up as maybe?'}) (input_keys={'question'})

In [49]:
from dspy.evaluate import SemanticF1

# Instantiate the metric.
metric = SemanticF1(decompositional=True)

# Produce a prediction from our `cot` module, using the `example` above as input.
pred = cot(**example.inputs())

# Compute the metric score for the prediction.
score = metric(example, pred)

print(f"Question: \t {example.question}\n")
print(f"Gold Response: \t {example.response}\n")
print(f"Predicted Response: \t {pred.response}\n")
print(f"Semantic F1 Score: {score:.2f}")

Question: 	 why are my text messages coming up as maybe?

Gold Response: 	 This is part of the Proactivity features new with iOS 9: It looks at info in emails to see if anyone with this number sent you an email and if it finds the phone number associated with a contact from your email, it will show you "Maybe". 

However, it has been suggested there is a bug in iOS 11.2 that can result in "Maybe" being displayed even when "Find Contacts in Other Apps" is disabled.

Predicted Response: 	 Your text messages are showing up as "maybe" because your messaging app is uncertain about the sender's identity. This typically happens when the sender's number is not saved in your contacts or if the message is coming from an unknown or unrecognized number. Check your contacts to see if the sender is listed, or consider saving their number to avoid this issue in the future.

Semantic F1 Score: 0.28


In [50]:
dspy.inspect_history(n=1)






[2025-05-18T10:44:40.956373]

System message:

Your input fields are:
1. `question` (str)
2. `ground_truth` (str)
3. `system_response` (str)
Your output fields are:
1. `reasoning` (str)
2. `ground_truth_key_ideas` (str): enumeration of key ideas in the ground truth
3. `system_response_key_ideas` (str): enumeration of key ideas in the system response
4. `discussion` (str): discussion of the overlap between ground truth and system response
5. `recall` (float): fraction (out of 1.0) of ground truth covered by the system response
6. `precision` (float): fraction (out of 1.0) of system response covered by the ground truth
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## ground_truth ## ]]
{ground_truth}

[[ ## system_response ## ]]
{system_response}

[[ ## reasoning ## ]]
{reasoning}

[[ ## ground_truth_key_ideas ## ]]
{ground_truth_key_ideas}

[[ ## system_response_key_ideas ## ]]
{system_response_k

In [55]:
# Define an evaluator that we can re-use.
evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24,
                         display_progress=True, display_table=2)

# Evaluate the Chain-of-Thought program.
evaluate(cot)

Average Metric: 39.11 / 63 (62.1%):  21%|█▋      | 63/300 [00:14<00:57,  4.12it/s]

2025/05/18 10:51:59 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 40.91 / 66 (62.0%):  22%|█▊      | 66/300 [00:15<01:05,  3.56it/s]

2025/05/18 10:52:00 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 48.20 / 77 (62.6%):  26%|██      | 77/300 [00:18<00:58,  3.78it/s]

...

2025/05/18 10:52:46 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 193.19 / 300 (64.4%): 100%|█████| 300/300 [01:05<00:00,  4.61it/s]

2025/05/18 10:52:50 INFO dspy.evaluate.evaluate: Average Metric: 193.1869318525495 / 300 (64.4%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],Choosing between C and C++ depends on the specific requirements of...,"Use C over C++ when you need a lightweight, procedural language wi...",✔️ [0.774]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",Storing images in a git repository depends on the context and the ...,It depends on the size and number of images. For small projects wi...,✔️ [0.750]


64.4

In [57]:
download("https://huggingface.co/dspy/cache/resolve/main/ragqa_arena_tech_corpus.jsonl")


In [59]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query

with open("ragqa_arena_tech_corpus.jsonl") as f:
    corpus = [ujson.loads(line)['text'][:max_characters] for line in f]
    print(f"Loaded {len(corpus)} documents. Will encode them below.")

len(corpus)

Loaded 28436 documents. Will encode them below.


28436

In [60]:
embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

Training a 32-byte FAISS index with 337 partitions, based on 28436 x 512-dim embeddings


In [64]:
p=search("what are high memory and low memory on linux?")
p.passages

['As far as I remember, High Memory is used for application space and Low Memory for the kernel. Advantage is that (user-space) applications cant access kernel-space memory.',
 'HIGHMEM is a range of kernels memory space, but it is NOT memory you access but its a place where you put what you want to access. A typical 32bit Linux virtual memory map is like: 0x00000000-0xbfffffff: user process (3GB) 0xc0000000-0xffffffff: kernel space (1GB) (CPU-specific vector and whatsoever are ignored here). Linux splits the 1GB kernel space into 2 pieces, LOWMEM and HIGHMEM. The split varies from installation to installation. If an installation chooses, say, 512MB-512MB for LOW and HIGH mems, the 512MB LOWMEM (0xc0000000-0xdfffffff) is statically mapped at the kernel boot time; usually the first so many bytes of the physical memory is used for this so that virtual and physical addresses in this range have a constant offset of, say, 0xc0000000. On the other hand, the latter 512MB (HIGHMEM) has no stat

In [65]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [66]:
rag = RAG()
rag(question="what are high memory and low memory on linux?")

Prediction(
    reasoning="High memory (HighMem) and low memory (LowMem) are segments of physical memory in Linux systems, particularly relevant in 32-bit architectures. Low memory is always mapped into the kernel's address space, allowing the kernel to access it directly at all times. High memory, on the other hand, is memory above a certain threshold (around 860MB in some systems) that is not permanently mapped into the kernel's address space. To access high memory, the kernel must explicitly map pages into its address space using functions like kmap, which makes high memory slower to access compared to low memory. User-space programs can directly access high memory, but the kernel must perform special steps to access it when needed. This division helps optimize memory management and system performance, especially when physical memory exceeds the kernel's direct mapping capacity.",
    response="High memory in Linux refers to the segment of physical memory that is not permanently map

In [67]:
evaluate(rag)


Average Metric: 23.26 / 35 (66.5%):  11%|▉       | 34/300 [00:15<00:50,  5.30it/s]

2025/05/18 11:18:38 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 59.64 / 84 (71.0%):  28%|██▏     | 84/300 [00:27<00:58,  3.69it/s]

2025/05/18 11:18:50 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 77.81 / 107 (72.7%):  36%|██▏   | 107/300 [00:32<00:36,  5.31it/s]

...

2025/05/18 11:19:41 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 218.15 / 300 (72.7%): 100%|█████| 300/300 [01:21<00:00,  3.68it/s]

2025/05/18 11:19:44 INFO dspy.evaluate.evaluate: Average Metric: 218.15210245486182 / 300 (72.7%)


,question,example_response,gold_doc_ids,reasoning,pred_response,SemanticF1
0,"when to use c over c++, and c++ over c?","If you are equally familiar with both C++ and C, it's advisable to...",[733],The context provides insights into the advantages and typical use ...,Use C over C++ primarily in embedded systems or environments where...,✔️ [0.500]
1,should images be stored in a git repository?,"One viewpoint expresses that there is no significant downside, esp...","[6253, 6254, 6275, 6278, 8215]",The context provides multiple perspectives on storing images in a ...,"Images can be stored in a git repository, but it is generally not ...",✔️ [0.750]


72.72

In [ ]:
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(RAG(), trainset=trainset,
                           max_bootstrapped_demos=3, max_labeled_demos=3,
                           requires_permission_to_run=False)

In [ ]:
evaluate(optimized_rag)

In [3]:
import random
import json
import asyncio

import pandas as pd
from collections import defaultdict
from datasets import load_dataset
from tqdm.asyncio import tqdm_asyncio
import pickle

from predictors import *
from llm import *
from prompts import *
from tools import *
from evaluations import *

dataset = load_dataset("gsm8k", "main")
training, validation = build_datasets(dataset, 1000, 0.7)


In [4]:
for x in training:
    x['answer']=int(x['answer'].replace(',', ''))

In [5]:
data = [dspy.Example(**d).with_inputs('question') for d in training]
len(data)


700

In [6]:
trainset, devset = data[:300], data[300:500]

In [32]:
class Solver(dspy.Module):
    def __init__(self):
        self.guess = dspy.ChainOfThought('question -> answer : int')
        self.critic = [dspy.ChainOfThought('question, last_reasoning, last_answer -> corrected_answer : int') for _ in range(3)]

    def forward(self, question):
        pred = self.guess(question=question)
        last_answer, last_reasoning = pred.answer, pred.reasoning
        for k in range(3):
            newpred = self.critic[k](question=question,
                                  last_reasoning=last_reasoning,
                                  last_answer=last_answer)
            last_answer, last_reasoning = newpred.corrected_answer, last_reasoning +" ==> "+ newpred.reasoning
        return dspy.Prediction(reasoning = last_reasoning, answer = last_answer)

solver = Solver()

In [35]:
solver(**data[0].inputs())

Prediction(
    reasoning='Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. ==> Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. ==> The problem states that Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. The calculation is correct, and the total clips sold is 72. ==> The problem states that Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add th

In [36]:
solver.guess(question=data[0].question)

Prediction(
    reasoning='Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72.',
    answer=72
)

In [37]:
def match(example, pred):
    return example.answer == pred.answer


evaluate = dspy.Evaluate(devset=devset, metric=match, num_threads=24,
                         display_progress=True, display_table=2)

In [38]:
evaluate(solver)

Average Metric: 36.00 / 38 (94.7%):  18%|█▍      | 37/200 [00:08<00:28,  5.72it/s]

2025/05/18 13:37:40 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 70.00 / 75 (93.3%):  38%|███     | 75/200 [00:14<00:21,  5.83it/s]

2025/05/18 13:37:46 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 71.00 / 76 (93.4%):  38%|███     | 75/200 [00:14<00:21,  5.83it/s]

2025/05/18 13:37:46 ERROR dspy.utils.parallelizer: Error for Example({'question': "Sam works at the Widget Factory, assembling Widgets. He can assemble 1 widget every 10 minutes. Jack from the loading dock can help assemble widgets when he doesn't have anything else to do. When he helps, they put together 2 complete widgets every 15 minutes. Recently the factory hired Tony to help assemble widgets. Being new to the job, he doesn't work as fast as Sam or Jack. Yesterday Sam worked for 6 hours before he had to leave work early for a dentist appointment. Jack was able to help out for 4 hours before he had to go back to the loading dock to unload a new shipment of widget materials. Tony worked the entire 8-hour shift. At the end of the day, they had completed 68 widgets. How long does it take Tony to assemble a Widget, in minutes?", 'thoughts': 'Sam completes a widget every 10 minutes.\nWhen Jack helps they finish 2 in 15 minutes. Sam has finished 1 widget and has begun working on another 

Average Metric: 174.00 / 199 (87.4%): 100%|█████| 200/200 [00:33<00:00,  5.98it/s]

2025/05/18 13:38:06 INFO dspy.evaluate.evaluate: Average Metric: 174.0 / 200 (87.0%)


,question,thoughts,example_answer,reasoning,pred_answer,match,answer
0,It costs $.10 to print one page. Jenny wants to print 7 copies of ...,Jenny needs to print 7*25= <<7*25=175>>175 pages. This will cost 1...,12.0,"First, calculate the cost to print 7 copies of a 25-page essay. Ea...",12.0,✔️ [True],NaN
1,240 sixth-grade students were asked about the number of novels the...,1/6 * 240 students = <<1/6*240=40>>40 students. So 40 students rea...,16.0,"First, find the number of students in each category based on the g...",16.0,✔️ [True],NaN


87.0

In [39]:
tp = dspy.MIPROv2(metric=match, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(Solver(), trainset=trainset,
                           max_bootstrapped_demos=0, max_labeled_demos=0,
                           requires_permission_to_run=False)

2025/05/18 13:38:20 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 57
minibatch: True
num_fewshot_candidates: 12
num_instruct_candidates: 6
valset size: 240

2025/05/18 13:38:20 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/18 13:38:20 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used for informing instruction proposal.

2025/05/18 13:38:20 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=12 sets of demonstrations...


Bootstrapping set 1/12
Bootstrapping set 2/12


 15%|██████▉                                       | 9/60 [00:48<04:34,  5.39s/it]
2025/05/18 13:39:08 INFO dspy.teleprompt.mipro_optimizer_v2: Error generating few-shot examples: match() takes 2 positional arguments but 3 were given
2025/05/18 13:39:08 INFO dspy.teleprompt.mipro_optimizer_v2: Running without few-shot examples.
2025/05/18 13:39:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/05/18 13:39:08 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.
2025/05/18 13:39:08 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing N=6 instructions...

2025/05/18 13:40:26 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/05/18 13:40:26 INFO dspy.teleprompt.mipro_optimizer_v2: 0: Given the fields `question`, produce the fields `answer`.

20

Average Metric: 6.00 / 7 (85.7%):   3%|▎          | 7/240 [00:03<00:56,  4.09it/s]

2025/05/18 13:40:30 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


...

2025/05/18 13:59:35 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 174.00 / 193 (90.2%):  80%|████ | 193/240 [00:15<00:03, 14.31it/s]

2025/05/18 13:59:39 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 216.00 / 239 (90.4%): 100%|█████| 240/240 [00:20<00:00, 11.55it/s]

2025/05/18 13:59:43 INFO dspy.evaluate.evaluate: Average Metric: 216.0 / 240 (90.0%)
2025/05/18 13:59:43 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [91.67, 91.25, 91.67, 90.0, 90.0, 90.83, 90.0, 90.83, 90.0, 90.83, 92.08, 90.83, 90.0]
2025/05/18 13:59:43 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 92.08
2025/05/18 13:59:43 INFO dspy.teleprompt.mipro_optimizer_v2: ========================
2025/05/18 13:59:43 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/05/18 13:59:43 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 92.08!


In [40]:
evaluate(optimized_rag)

Average Metric: 36.00 / 37 (97.3%):  18%|█▍      | 37/200 [00:05<00:25,  6.33it/s]

2025/05/18 13:59:49 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 71.00 / 75 (94.7%):  38%|███     | 75/200 [00:08<00:10, 12.30it/s]

2025/05/18 13:59:52 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 72.00 / 76 (94.7%):  38%|███     | 75/200 [00:08<00:10, 12.30it/s]

...

2025/05/18 14:00:07 WARNING dspy.adapters.json_adapter: Failed to use structured output format. Falling back to JSON mode. Error: 'type' object is not iterable


Average Metric: 175.00 / 199 (87.9%): 100%|█████| 200/200 [00:26<00:00,  7.55it/s]

2025/05/18 14:00:10 INFO dspy.evaluate.evaluate: Average Metric: 175.0 / 200 (87.5%)


,question,thoughts,example_answer,reasoning,pred_answer,match,answer
0,It costs $.10 to print one page. Jenny wants to print 7 copies of ...,Jenny needs to print 7*25= <<7*25=175>>175 pages. This will cost 1...,12.0,"First, calculate the cost to print 7 copies of a 25-page essay. Ea...",12.0,✔️ [True],NaN
1,240 sixth-grade students were asked about the number of novels the...,1/6 * 240 students = <<1/6*240=40>>40 students. So 40 students rea...,16.0,"First, find the number of students in each category based on the g...",16.0,✔️ [True],NaN


87.5

In [41]:
optimized_rag(**data[0].inputs())

Prediction(
    reasoning='Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. ==> Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. ==> Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. ==> Natalia sold 48 clips in April. In May, she sold half as many clips, which is 48 divided by 2, resulting in 24 clips. To find the total number of clips sold in April and May, add the clips sold in both months: 48 + 24 = 72. The calculation is correct, and the total is 72 clips.',
    answ

In [42]:
dspy.inspect_history(n=1)






[2025-05-18T14:00:12.984661]

System message:

Your input fields are:
1. `question` (str)
2. `last_reasoning` (str)
3. `last_answer` (str)
Your output fields are:
1. `reasoning` (str)
2. `corrected_answer` (int)
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## last_reasoning ## ]]
{last_reasoning}

[[ ## last_answer ## ]]
{last_answer}

[[ ## reasoning ## ]]
{reasoning}

[[ ## corrected_answer ## ]]
{corrected_answer}        # note: the value you produce must be a single int value

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `question`, `last_reasoning`, `last_answer`, produce the fields `corrected_answer`.


User message:

[[ ## question ## ]]
Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?

[[ ## last_reasoning ## ]]
Natalia sold 48 c